In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Model

In [9]:
# first step we define the image path
content_path="assets/content_file.png"
style_path="assets/style_file.png"

In [10]:
## this function is create to get the image from the path and process it before working with it 
def process_image(path_of_image):
    image = load_img(path_of_image)
    image = img_to_array(image)
    image = preprocess_input(image)
    image = np.expand_dims(image,0)
    return image

In [11]:
def deprocess_image(image):
    image [:,:,0]+=103.939
    image [:,:,1]+=116.779
    image [:,:,2]+=123.68
    image = image[:,:,::-1]
    image = np.clip(image,0,255).astype('uint8')
    return image

In [12]:
def display(image):
    if(len(image.shape)==4):
        image = np.squeeze(image,axis=0)
    image = deprocess_image(image)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    return 

In [ ]:
content_image = process_image(content_path)
style_image = process_image(style_path)

display(content_image)
display(style_image)


In [ ]:
def gram_matrix(A):
    channels = int(A.shape[-1])
    a = tf.reshape(A,[-1, channels])
    gram = tf.matmul(a,a,transpose_a=True)
    return gram / tf.cast(tf.shape(a)[0],tf.float32)
weight_of_layer = 1. / len(style_models)

In [15]:
def content_loss (content , generated):
    map_of_content_image = content_model(content)
    map_of_generated_image = content_model(generated)
    loss = tf.reduce_mean(tf.square(map_of_content_image-map_of_generated_image))
    return loss

In [16]:
def style_cost(style,generated):
    J_style = 0
    for style_model in style_models:
        map_of_style_image = style_model(style)
        map_of_generated_image = style_model(generated)
        gram_matrix_style_image = gram_matrix(map_of_style_image)
        gram_matrix_generated_image = gram_matrix(map_of_generated_image)
        content_cost = tf.reduce_mean(tf.square(gram_matrix_style_image-gram_matrix_generated_image))
        J_style += content_cost * weight_of_layer
    return J_style

In [17]:
vgg = VGG19(include_top=False, weights='imagenet')
content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']


In [18]:
def get_model():
    outputs = [vgg.get_layer(name).output for name in style_layers]
    outputs.append(vgg.get_layer(content_layer).output)
    model = tf.keras.Model([vgg.input], outputs)
    return model

In [19]:
def get_features(image, model):
    outputs = model(image)
    style_features = outputs[:-1]
    content_feature = outputs[-1]
    return style_features, content_feature

In [20]:
alpha = 1e3
beta = 1e-2

def compute_total_loss(content_image, style_image, generated_image, model):
    style_features, content_feature = get_features(content_image, model)
    generated_style_features, generated_content_feature = get_features(generated_image, model)
    
    J_content = content_loss(content_feature, generated_content_feature)
    J_style = style_cost(style_features, generated_style_features)
    J_total = alpha * J_content + beta * J_style
    return J_total

In [21]:
input_image = tf.Variable(process_image('assets/content_file.png'), dtype=tf.float32)
style_image = process_image('assets/style_file.png')
content_image = process_image('assets/content_file.png')

model = get_model()

optimizer = tf.optimizers.Adam(learning_rate=0.02)

In [ ]:
def train_step(generated_image, content_image, style_image, model):
    with tf.GradientTape() as tape:
        loss = compute_total_loss(content_image, style_image, generated_image, model)
    gradients = tape.gradient(loss, generated_image)
    optimizer.apply_gradients([(gradients, generated_image)])
    generated_image.assign(tf.clip_by_value(generated_image, 0.0, 255.0))
    return loss

epochs = 1000

for epoch in range(epochs):
    loss = train_step(input_image, content_image, style_image, model)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")
        display(input_image.numpy())

final_image = input_image.numpy()
final_image = deprocess_image(final_image[0])
Image.fromarray(final_image).save('stylized_image.jpg')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Function to preprocess images for VGG19
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# Function to deprocess images from VGG19 format
def deprocess_image(img):
    img = img.reshape((img.shape[1], img.shape[2], 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

# Function to load VGG19 model with specific outputs for style and content
def get_model():
    vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    style_outputs = [vgg.get_layer(name).output for name in ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']]
    content_outputs = [vgg.get_layer('block4_conv2').output]
    model_outputs = style_outputs + content_outputs
    return tf.keras.models.Model(vgg.input, model_outputs)

# Function to calculate style loss
def style_loss(style, combination):
    style_outputs = get_model()(style)
    combination_outputs = get_model()(combination)
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[i] - combination_outputs[i])**2) for i in range(len(style_outputs))])
    return style_loss

# Function to calculate content loss
def content_loss(content, combination):
    content_outputs = get_model()(content)[-1]
    combination_outputs = get_model()(combination)[-1]
    content_loss = tf.reduce_mean((content_outputs - combination_outputs)**2)
    return content_loss

# Parameters and hyperparameters
content_image_path = 'assets/content_file.png'
style_image_path = 'assets/style_file.png'

learning_rate = 10.0
epochs = 1000
content_weight = 1e3
style_weight = 1e-2

# Load and preprocess images
content_image = preprocess_image(content_path)
style_image = preprocess_image(style_path)
initial_generated_image = preprocess_image(content_path)
generated_image = tf.Variable(initial_generated_image, dtype=tf.float32)

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Function to apply gradients and optimize the generated image
@tf.function()
def train_step(image, content_image, style_image, content_weight, style_weight, optimizer):
    with tf.GradientTape() as tape:
        tape.watch(image)
        content_loss_value = content_loss(content_image, image)
        style_loss_value = style_loss(style_image, image)
        total_loss = content_weight * content_loss_value + style_weight * style_loss_value
    gradients = tape.gradient(total_loss, image)
    optimizer.apply_gradients([(gradients, image)])
    image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=255.0))
    return total_loss

# Training loop
for epoch in range(epochs):
    total_loss = train_step(generated_image, content_image, style_image, content_weight, style_weight, optimizer)
    if epoch % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss.numpy()}")

# Convert generated image back to PIL format for saving
final_image = deprocess_image(generated_image.numpy())

# Save stylized image
Image.fromarray(final_image).save('stylized_image.png')

# Display final stylized image
plt.imshow(final_image)
plt.axis('off')
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Function to preprocess images for VGG19
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# Function to deprocess images from VGG19 format
def deprocess_image(img):
    img = img.reshape((img.shape[1], img.shape[2], 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

# Function to load VGG19 model with specific outputs for style and content
def get_model():
    vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    style_outputs = [vgg.get_layer(name).output for name in ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']]
    content_outputs = [vgg.get_layer('block4_conv2').output]
    model_outputs = style_outputs + content_outputs
    return tf.keras.models.Model(vgg.input, model_outputs)

# Function to calculate style loss
def style_loss(style, combination):
    style_outputs = get_model()(style)
    combination_outputs = get_model()(combination)
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[i] - combination_outputs[i])**2) for i in range(len(style_outputs))])
    return style_loss

# Function to calculate content loss
def content_loss(content, combination):
    content_outputs = get_model()(content)[-1]
    combination_outputs = get_model()(combination)[-1]
    content_loss = tf.reduce_mean((content_outputs - combination_outputs)**2)
    return content_loss

# Parameters and hyperparameters
content_path = 'assets/content_file.png'
style_path = 'assets/style_file.png'

learning_rate = 10.0
epochs = 1000
content_weight = 1e3
style_weight = 1e-2

# Load and preprocess images
content_image = preprocess_image(content_path)
style_image = preprocess_image(style_path)
initial_generated_image = preprocess_image(content_path)
generated_image = tf.Variable(initial_generated_image, dtype=tf.float32)

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Function to apply gradients and optimize the generated image
@tf.function()
def train_step(image, content_image, style_image, content_weight, style_weight, optimizer):
    with tf.GradientTape() as tape:
        tape.watch(image)
        content_loss_value = content_loss(content_image, image)
        style_loss_value = style_loss(style_image, image)
        total_loss = content_weight * content_loss_value + style_weight * style_loss_value
    gradients = tape.gradient(total_loss, image)
    optimizer.apply_gradients([(gradients, image)])
    image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=255.0))
    return total_loss

# Training loop
for epoch in range(epochs):
    total_loss = train_step(generated_image, content_image, style_image, content_weight, style_weight, optimizer)
    if epoch % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss.numpy()}")

# Convert generated image back to PIL format for saving
final_image = deprocess_image(generated_image.numpy())

# Save stylized image
Image.fromarray(final_image).save('stylized_image.png')

# Display final stylized image
plt.imshow(final_image)
plt.axis('off')
plt.show()
